## <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [1]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()        

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://2176a0f0-3503-11ed-be64-b1a4781e5f0a.cloud.hopsworks.ai/p/128
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27;"> 🪝 Feature View and Training Dataset Retrieval </span>

In [2]:
feature_view = fs.get_feature_view(
    name = 'titanic_fv',
    version = 1
)

In [3]:
X_train,y_train, X_val,y_val, X_test,y_test = feature_view.get_train_validation_test_split(2)
for x in [X_train, X_test, X_val]:
    x = x.drop("passengerid",axis=1)

X_train.head()

,passengerid,sex,age,pclass,sibsp,parch,fare,embarked
0,1,1,-0.587421,0.820482,0.492906,-0.464282,0.014151,2
1,2,0,0.633474,-1.579331,0.492906,-0.464282,0.139136,1
2,3,0,-0.282197,0.820482,-0.478867,-0.464282,0.015469,2
3,4,0,0.404556,-1.579331,0.492906,-0.464282,0.103644,2
4,5,1,0.404556,0.820482,-0.478867,-0.464282,0.015713,2


In [4]:
y_train.head()

,survived
0,0
1,1
2,1
3,1
4,0


## <span style='color:#ff5f27'> 📝 Imports

In [5]:
import numpy as np

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset,Dataset

## <span style="color:#ff5f27;"> 🤖 Model Building </span>

#### <span style="color:#ff5f27;"> 👩🏻‍🔬 PyTorch Model </span>

In [6]:
class PyTorchModel(nn.Module):
    def __init__(self,input_size,output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return F.sigmoid(x)

In [7]:
model = PyTorchModel(X_train.shape[1],y_train.nunique()[0])
model

PyTorchModel(
  (fc1): Linear(in_features=8, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

#### <span style="color:#ff5f27;"> 🧬 DataFrames to Tensors</span>

In [8]:
def to_tensor(X,y):
    X = torch.tensor(X.values.astype(np.float32)) 
    y = torch.tensor(y.values.astype(np.float32)) 
    return X,y

In [9]:
X_train_t,y_train_t = to_tensor(X_train,y_train)
X_val_t,y_val_t = to_tensor(X_val,y_val)
X_test_t,y_test_t = to_tensor(X_test,y_test)

#### <span style="color:#ff5f27;"> 🏋️‍♀️ Model Training </span>

In [10]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

In [11]:
for epoch in range(50):
    preds = model(X_train_t)
    loss_value = loss(preds, y_train_t)
    
    optimizer.zero_grad()
    loss_value.backward()        
    optimizer.step()

val_preds = model.forward(X_val_t)            
(val_preds.argmax(dim=1) == y_val_t).float().mean()    

tensor(0.5773)

## <span style="color:#ff5f27;"> 🚀 Model Deployment </span>

## !! We have a problem with online-enabled Featre Groups (already reported on Jira), so we cannot proceed with Deployments.

In [12]:
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


In [13]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

model_schema.to_dict()

{'input_schema': {'columnar_schema': [{'name': 'passengerid', 'type': 'int64'},
   {'name': 'sex', 'type': 'int64'},
   {'name': 'age', 'type': 'float64'},
   {'name': 'pclass', 'type': 'float64'},
   {'name': 'sibsp', 'type': 'float64'},
   {'name': 'parch', 'type': 'float64'},
   {'name': 'fare', 'type': 'float64'},
   {'name': 'embarked', 'type': 'int64'}]},
 'output_schema': {'columnar_schema': [{'name': 'survived', 'type': 'int64'}]}}

In [14]:
import joblib

model_dir = "./model"
torch.save(model, model_dir)

model_hops = mr.sklearn.create_model(
    name = "pytorch_model",
    input_example = X_train.sample(),
    model_schema = model_schema
)

model_hops.save(model_dir)

  0%|          | 0/6 [00:00<?, ?it/s]

Model created, explore it at https://2176a0f0-3503-11ed-be64-b1a4781e5f0a.cloud.hopsworks.ai/p/128/models/pytorch_model/2


Model(name: 'pytorch_model', version: 2)

In [15]:
torch.load('./model')

PyTorchModel(
  (fc1): Linear(in_features=8, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [27]:
%%writefile predict_example.py
import os
import hsfs
import torch
import numpy as np

class Predict(object):

    def __init__(self):
        """ Initializes the serving state, reads a trained model"""        
        # load the trained model
        self.model = torch.load(os.environ["ARTIFACT_FILES_PATH"] + '/model').eval()
        print("Initialization Complete")

    def predict(self, inputs):
        """ Serves a prediction request usign a trained model"""
        return self.model(np.array(inputs).reshape(1, -1)).tolist()

Overwriting predict_example.py


In [23]:
import os
dataset_api = project.get_dataset_api()

uploaded_file_path = dataset_api.upload("predict_example.py", "Models", overwrite=True)
predictor_script_path = os.path.join("/Projects", project.name, uploaded_file_path)

Uploading: 0.000%|          | 0/504 elapsed<00:00 remaining<?

In [24]:
# Use the model name from the previous notebook.
model = mr.get_model("pytorch_model", version = 1)

# Give it any name you want
deployment = model.deploy(
    name="pytorchmodeldeploy", 
    model_server="PYTHON",
    #serving_tool='KSERVE',
    script_file=predictor_script_path
)

RestAPIError: Metadata operation error: (url: https://hopsworks.glassfish.service.consul:8182/hopsworks-api/api/project/128/serving). Server response: 
HTTP code: 400, HTTP reason: Bad Request, error code: 240017, error msg: Model path does not have a valid file structure, user msg: Model path requires either a python script or model file (i.e., joblib or pickle files)

In [28]:
# Use the model name from the previous notebook.
model = mr.get_model("pytorch_model", version = 1)

# Give it any name you want
deployment = model.deploy(
    name="pytorchmodeldeploy", 
    model_server="torch",
    serving_tool='KSERVE',
    script_file=predictor_script_path
)

ValueError: Model server 'torch' is not valid. Possible values are 'PYTHON, TENSORFLOW_SERVING'

---